In [1]:
import pandas as pd

# Process Data

In [2]:
#!/home/lgunthermann/miniconda3/envs/discushack22/bin/python3 process_csv.py

In [3]:
data_coding = pd.read_csv('data/processed/coding_both.csv')
data_invoice = pd.read_csv('data/processed/invoices.csv')

In [4]:
from IPython.display import display, HTML
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

def print_positions(party,supplier):
    print("Invoices:")
    invoices = data_invoice.loc[ (data_invoice['RegulatedEntityName'] == party) & (data_invoice['SupplierName'] == supplier) ]
    display(invoices)
    print("Sum TotalExpenditure:",invoices['TotalExpenditure'].sum())
    print("\nCoding:")
    display(data_coding.loc[ (data_coding['Party'] == party) & (data_coding['Supplier'] == supplier) ])

In [5]:
#display(data_coding)

# Data Preview

In [6]:
data_coding.head()


,ID,Supplier,Party,Total Spend,Expense Category,Transport,Miscellaneous,Catering,Accommodation,Expenses claimed by provider,Completely unclear,Ambiguous,Advertising and Press,Merchandise,Newspaper or magazine advertising,Radio advertising,Social media advertising,Online advertising,Other forms of advertising,PR,Campaign materials,Design services,Campaign material printing,Postage,Infrastructure and equipment,Telecommunications services,Physical Security,Event costs/Venue hire,Mobile application services,Email services,Website services,Search Engine Optimization,Recruitment services/staffing costs,Creative content,Translation/Braile/British Sign Language services,Campaign activity,GOTV,Fundraising,Data and infrastructure,Campaign database or CRM,Data Services and analysis,IT infrastructure and support,Office supplies,Production Services,Video editing/production,Audio editing/production,Photos editing/production,Consultancy,Communication consultants,Design consultants,Ad strategy and consultancy,Social media strategy and consultancy,Data consultancy,Legal advice,Research,Polling,Focus groups,Ordinance survey data,Message testing,Archival research,Other forms of research,Social/Digital listening,Phone banking
0,0,Abi Bell,Liberal Democrats,1936.07,Transport,0.0,0.0,0.0,0.0,0.0,1936.07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,Abigail Baker,Liberal Democrats,748.80,Transport,0.0,0.0,0.0,0.0,0.0,748.80,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,Adam Prices,Plaid Cymru,275.85,Transport,0.0,0.0,0.0,0.0,0.0,275.85,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,Adam Williams,Liberal Democrats,484.06,Transport,0.0,0.0,0.0,0.0,0.0,484.06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,Addison Lee,Conservative Party,22648.35,Transport,0.0,0.0,0.0,0.0,0.0,22648.35,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
data_invoice.head()

,ECRef,RegulatedEntityName,ReportingPeriodName,RegulatedEntityType,TotalExpenditure,DateIncurred,ExpenseCategoryName,SupplierName,FullAddress,AmountInEngland,AmountInScotland,AmountInWales,AmountInNorthernIreland,DateOfClaimForPayment,DatePaid,RedactedSupportingInvoiceId,RegulatedEntityId,SupplierId,JointCampaignName,UnregisteredCampaignerName,CampaigningName,IsOutsideSection75
0,SP0555253,Plaid Cymru,UK Parliamentary general election 12/12/2019,Political Party,200.40,03/01/2019,Unsolicited material to electors,Low Cost Printing,"Precision House, Pentrpoeth Rd, Swansea, SA6 6AA, United Kingdom",£0.00,£0.00,£200.40,£0.00,NaN,03/01/2019,74031,77,2276,NaN,NaN,NaN,False
1,SP0541026,Liberal Democrats,UK Parliamentary general election 12/12/2019,Political Party,60.00,01/04/2019,Advertising,Rayleigh Town Council,"The Pavilion, KGV Playing Field, Bull Lane, Rayleigh, SS6 8JD, United Kingdom",£60.00,£0.00,£0.00,£0.00,01/04/2019,10/05/2019,0,90,68727,NaN,NaN,NaN,False
2,SP0541025,Liberal Democrats,UK Parliamentary general election 12/12/2019,Political Party,36.00,17/06/2019,Market research/canvassing,Rochford District Council,"Council Offices, South Street, ROCHFORD, SS4 1BW, United Kingdom",£36.00,£0.00,£0.00,£0.00,17/06/2019,04/07/2019,0,90,76767,NaN,NaN,NaN,False
3,SP0541024,Liberal Democrats,UK Parliamentary general election 12/12/2019,Political Party,9.99,07/01/2019,Advertising,RelyOn IT (UK),"1386 London Rd, Leigh-on-Sea, SS9 2UJ, United Kingdom",£9.99,£0.00,£0.00,£0.00,07/01/2019,10/01/2019,0,90,76765,NaN,NaN,NaN,False
4,SP0541023,Liberal Democrats,UK Parliamentary general election 12/12/2019,Political Party,72.00,18/01/2019,Unsolicited material to electors,Off-Tech,"1 Brodie Road, ENFIELD, ZZ0 0ZZ, United Kingdom",£72.00,£0.00,£0.00,£0.00,18/01/2019,18/02/2019,0,90,76766,NaN,NaN,NaN,False


# Test

In [12]:
def get_accounted_output(name,accounted,unaccounted,overaccounted):
    acc_total = sum([accounted,unaccounted,overaccounted])
    return '{:<55s}'.format(name+':') \
            + '{:<40s}'.format('accounted = £'+'{0:.2f}'.format(accounted)+' ('+'{0:.1f}'.format((100*accounted/acc_total) if acc_total>0 else 0)+'%)') \
            + '{:<40s}'.format('| un-accounted = £'+'{0:.2f}'.format(unaccounted)+' ('+'{0:.1f}'.format((100*unaccounted/acc_total) if acc_total>0 else 0)+'%)') \
            + '{:<40s}'.format('| over-accounted = £'+'{0:.2f}'.format(overaccounted)+' ('+'{0:.1f}'.format((100*overaccounted/acc_total) if acc_total>0 else 0)+'%)')+'\n'


output=''
parties = data_coding['Party'].unique()
categories = data_coding['Expense Category'].unique()

total_accounted = 0
total_unaccounted = 0
total_overaccounted = 0

positions = []

for party in parties:
    print(f"Party: {party}")
    output += '\n####################################################################################\n  '+party+'\n####################################################################################\n\n'
    sub_party_coding = data_coding[ data_coding['Party']==party ]
    sub_party_invoice = data_invoice[ data_invoice['RegulatedEntityName']==party ]
    
    party_accounted = 0
    party_unaccounted = 0
    party_overaccounted = 0
    
    for category in categories:
        sub_cat_coding = sub_party_coding[ sub_party_coding['Expense Category']==category ]
        sub_cat_invoice = sub_party_invoice[ sub_party_invoice['ExpenseCategoryName']==category ]
        
        accounted = 0
        unaccounted = 0
        overaccounted = 0
        
        for idx, row in sub_cat_coding.iterrows():
            supplier = row['Supplier']
            spent = row['Total Spend']
            
            sub_pos_invoice = sub_cat_invoice[ sub_cat_invoice['SupplierName']==supplier ]
            filed = sub_pos_invoice['TotalExpenditure'].sum()
            
            sup_accounted = min(spent,filed)
            
            if spent-filed > 0.1:
                sup_unaccounted = spent-filed
            else:
                sup_unaccounted = 0
             
            if filed-spent > 0.1:
                sup_overaccounted = filed-spent
            else:
                sup_overaccounted = 0
            
            positions.append([party, supplier, category, sup_accounted, sup_unaccounted, sup_overaccounted])
            
            sup_sum = sup_accounted+sup_unaccounted+sup_overaccounted
            
            if sup_unaccounted > 0 or sup_overaccounted > 0:
                print(f"{party} - {category} - {supplier}: Accounted = £{sup_accounted:.2f} ({(100*sup_accounted/sup_sum):.2f}%) "
                      + f"| Unaccounted = £{sup_unaccounted:.2f} ({(100*sup_unaccounted/sup_sum):.2f}%) | Over-Accounted = £{sup_overaccounted:.2f} ({(100*sup_overaccounted/sup_sum):.2f}%)")
            
            accounted += sup_accounted
            unaccounted += sup_unaccounted
            overaccounted += sup_overaccounted
        
        party_accounted += accounted
        party_unaccounted += unaccounted
        party_overaccounted += overaccounted
    
        output += get_accounted_output(' Category '+category,accounted,unaccounted,overaccounted)
    output += '-'*170+'\n'
    output += get_accounted_output(' Total',party_accounted,party_unaccounted,party_overaccounted)
    
    total_accounted += party_accounted
    total_unaccounted += party_unaccounted
    total_overaccounted += party_overaccounted

df = pd.DataFrame(positions, columns=['Party', 'Supplier', 'Expense Category', 'Accounted', 'Unaccounted', 'Over-Accounted']) 



Party: Liberal Democrats
Liberal Democrats - Transport - Airbnb: Accounted = £5395.92 (88.30%) | Unaccounted = £0.00 (0.00%) | Over-Accounted = £714.70 (11.70%)
Liberal Democrats - Transport - Airbnb: Accounted = £714.70 (11.70%) | Unaccounted = £0.00 (0.00%) | Over-Accounted = £5395.92 (88.30%)
Liberal Democrats - Transport - Sainsburys: Accounted = £0.00 (0.00%) | Unaccounted = £522.05 (100.00%) | Over-Accounted = £0.00 (0.00%)
Liberal Democrats - Advertising - Prater Raines: Accounted = £20764.33 (94.47%) | Unaccounted = £1215.22 (5.53%) | Over-Accounted = £0.00 (0.00%)
Liberal Democrats - Advertising - Telebells Call Centre: Accounted = £6571.50 (40.68%) | Unaccounted = £9584.60 (59.32%) | Over-Accounted = £0.00 (0.00%)
Liberal Democrats - Advertising - Bath and North East Somerset Liberal Democrats: Accounted = £1538.46 (39.26%) | Unaccounted = £2380.03 (60.74%) | Over-Accounted = £0.00 (0.00%)
Liberal Democrats - Advertising - Facebook: Accounted = £1316737.93 (99.97%) | Unaccoun

In [13]:
print(get_accounted_output('Overall',total_accounted,total_unaccounted,total_overaccounted))
print(output)

Overall:                                               accounted = £33075023.03 (66.3%)        | un-accounted = £16485732.50 (33.1%)   | over-accounted = £299779.00 (0.6%)    


####################################################################################
  Liberal Democrats
####################################################################################

 Category Transport:                                   accounted = £265743.17 (97.6%)          | un-accounted = £522.05 (0.2%)         | over-accounted = £6110.62 (2.2%)      
 Category Advertising:                                 accounted = £2599322.26 (30.4%)         | un-accounted = £5961105.59 (69.6%)    | over-accounted = £0.00 (0.0%)         
 Category Unsolicited material to electors:            accounted = £3535320.69 (99.5%)         | un-accounted = £18432.91 (0.5%)       | over-accounted = £0.00 (0.0%)         
 Category Campaign broadcasts:                         accounted = £78305.40 (100.0%)          | un-acc

In [14]:
df = df.sort_values(by=['Unaccounted'], ascending=False)
df.head()

,Party,Supplier,Expense Category,Accounted,Unaccounted,Over-Accounted
146,Liberal Democrats,Postal Choices,Advertising,653.84,3027667.02,0.0
602,Conservative Party,Whistl,Unsolicited material to electors,0.00,2436522.48,0.0
671,Conservative Party,Paragon Customer Communications,Manifesto or Referendum material,15852.10,1323687.77,0.0
886,Labour Party,Sterling,Advertising,354.00,1318872.63,0.0
123,Liberal Democrats,Royal Mail,Advertising,524502.11,842103.88,0.0


In [15]:
print_positions(party='Liberal Democrats',supplier='Postal Choices')

Invoices:


,ECRef,RegulatedEntityName,ReportingPeriodName,RegulatedEntityType,TotalExpenditure,DateIncurred,ExpenseCategoryName,SupplierName,FullAddress,AmountInEngland,AmountInScotland,AmountInWales,AmountInNorthernIreland,DateOfClaimForPayment,DatePaid,RedactedSupportingInvoiceId,RegulatedEntityId,SupplierId,JointCampaignName,UnregisteredCampaignerName,CampaigningName,IsOutsideSection75
32,SP0540995,Liberal Democrats,UK Parliamentary general election 12/12/2019,Political Party,9399.01,09/12/2019,Unsolicited material to electors,Postal Choices,"Unit G8, Bolingbroke Way, Patchway, Bristol, BS34 6FE, United Kingdom","£9,399.01",£0.00,£0.00,£0.00,09/12/2019,30/12/2019,71669,90,68100,NaN,NaN,NaN,False
33,SP0540994,Liberal Democrats,UK Parliamentary general election 12/12/2019,Political Party,9399.01,09/12/2019,Unsolicited material to electors,Postal Choices,"Unit G8, Bolingbroke Way, Patchway, Bristol, BS34 6FE, United Kingdom","£9,399.01",£0.00,£0.00,£0.00,09/12/2019,30/12/2019,71668,90,68100,NaN,NaN,NaN,False
34,SP0540993,Liberal Democrats,UK Parliamentary general election 12/12/2019,Political Party,5276.49,02/12/2019,Unsolicited material to electors,Postal Choices,"Unit G8, Bolingbroke Way, Patchway, Bristol, BS34 6FE, United Kingdom","£5,276.49",£0.00,£0.00,£0.00,02/12/2019,23/12/2019,71667,90,68100,NaN,NaN,NaN,False
35,SP0540992,Liberal Democrats,UK Parliamentary general election 12/12/2019,Political Party,8129.63,29/11/2019,Unsolicited material to electors,Postal Choices,"Unit G8, Bolingbroke Way, Patchway, Bristol, BS34 6FE, United Kingdom","£8,129.63",£0.00,£0.00,£0.00,29/11/2019,20/12/2019,71666,90,68100,NaN,NaN,NaN,False
208,SP0540819,Liberal Democrats,UK Parliamentary general election 12/12/2019,Political Party,1295.00,02/12/2019,Unsolicited material to electors,Postal Choices,"Unit G8, Bolingbroke Way, Patchway, Bristol, BS34 6FE, United Kingdom","£1,295.00",£0.00,£0.00,£0.00,02/12/2019,23/12/2019,71582,90,68100,NaN,NaN,NaN,False
265,SP0540762,Liberal Democrats,UK Parliamentary general election 12/12/2019,Political Party,2946.33,06/11/2019,Unsolicited material to electors,Postal Choices,"Unit G8, Bolingbroke Way, Patchway, Bristol, BS34 6FE, United Kingdom","£2,946.33",£0.00,£0.00,£0.00,24/12/2019,24/12/2019,71555,90,68100,NaN,NaN,NaN,False
376,SP0540651,Liberal Democrats,UK Parliamentary general election 12/12/2019,Political Party,982.99,09/12/2019,Unsolicited material to electors,Postal Choices,"Unit G8, Bolingbroke Way, Patchway, Bristol, BS34 6FE, United Kingdom",£982.99,£0.00,£0.00,£0.00,09/12/2019,30/12/2019,71488,90,68100,NaN,NaN,NaN,False
387,SP0540640,Liberal Democrats,UK Parliamentary general election 12/12/2019,Political Party,3891.39,29/11/2019,Unsolicited material to electors,Postal Choices,"Unit G8, Bolingbroke Way, Patchway, Bristol, BS34 6FE, United Kingdom","£3,891.39",£0.00,£0.00,£0.00,29/11/2019,23/12/2019,71477,90,68100,NaN,NaN,NaN,False
413,SP0540614,Liberal Democrats,UK Parliamentary general election 12/12/2019,Political Party,1282.78,30/11/2019,Unsolicited material to electors,Postal Choices,"Unit G8, Bolingbroke Way, Patchway, Bristol, BS34 6FE, United Kingdom","£1,282.78",£0.00,£0.00,£0.00,09/12/2019,30/12/2019,71459,90,68100,NaN,NaN,NaN,False
416,SP0540611,Liberal Democrats,UK Parliamentary general election 12/12/2019,Political Party,954.00,30/11/2019,Unsolicited material to electors,Postal Choices,"Unit G8, Bolingbroke Way, Patchway, Bristol, BS34 6FE, United Kingdom",£954.00,£0.00,£0.00,£0.00,09/12/2019,30/12/2019,71456,90,68100,NaN,NaN,NaN,False


Sum TotalExpenditure: 3028320.86

Coding:


,ID,Supplier,Party,Total Spend,Expense Category,Transport,Miscellaneous,Catering,Accommodation,Expenses claimed by provider,Completely unclear,Ambiguous,Advertising and Press,Merchandise,Newspaper or magazine advertising,Radio advertising,Social media advertising,Online advertising,Other forms of advertising,PR,Campaign materials,Design services,Campaign material printing,Postage,Infrastructure and equipment,Telecommunications services,Physical Security,Event costs/Venue hire,Mobile application services,Email services,Website services,Search Engine Optimization,Recruitment services/staffing costs,Creative content,Translation/Braile/British Sign Language services,Campaign activity,GOTV,Fundraising,Data and infrastructure,Campaign database or CRM,Data Services and analysis,IT infrastructure and support,Office supplies,Production Services,Video editing/production,Audio editing/production,Photos editing/production,Consultancy,Communication consultants,Design consultants,Ad strategy and consultancy,Social media strategy and consultancy,Data consultancy,Legal advice,Research,Polling,Focus groups,Ordinance survey data,Message testing,Archival research,Other forms of research,Social/Digital listening,Phone banking
1094,1094,Postal Choices,Liberal Democrats,3028320.86,Advertising,0.0,0.0,0.0,0.0,0.0,645.43,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13048.4,3014422.09,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,204.94,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
